In [1]:
import os
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
#os.getenv('OPENAI_API_KEY')

In [4]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
client = OpenAI(max_tokens=50)

### Individual Chain

In [5]:
prompt = PromptTemplate.from_template("Suggest me a good company name making {product}")
chain = LLMChain(llm=client, prompt=prompt, verbose=True)
chain.invoke("footballs")

/home/lal/dev/practice/langchain-basics-using-pinecone-chromadb-openai/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Suggest me a good company name making footballs

> Finished chain.


{'product': 'footballs',
 'text': '\n\n1. GoalMaster\n2. KickNation\n3. BallBlitz\n4. ScoreStrike\n5. FieldFever\n6. ProPlay\n7. NetNation\n8. Touchline Traders\n9. The Football Factory'}

In [6]:
chain.invoke("orange juice")



> Entering new LLMChain chain...
Prompt after formatting:
Suggest me a good company name making orange juice

> Finished chain.


{'product': 'orange juice',
 'text': '\n\n1. ZestyOrange Co.\n2. Squeeze & Savor\n3. Freshly Pressed\n4. Orange Crush Co.\n5. Citrus Bliss\n6. Juicy Delight Co.\n7. Sunny Sips\n8'}

### Multiple Chains using Simple Sequential Chaining

In [7]:
from langchain.chains import SimpleSequentialChain
client = OpenAI(max_tokens=100)

In [8]:
company_name_prompt = PromptTemplate.from_template("Suggest me a company name for making {product}")
company_name_chain = LLMChain(llm=client, prompt=company_name_prompt)

In [9]:
company_strategies_prompt = PromptTemplate.from_template("Suggest few strategies for the this company {company}")
company_strategies_chain = LLMChain(llm=client, prompt=company_strategies_prompt)

In [10]:
chain = SimpleSequentialChain(chains=[company_name_chain, company_strategies_chain])

In [11]:
print(chain.invoke("headphones"))

{'input': 'headphones', 'output': "\n\n1. Expand product offerings: Soundwave Audio can consider expanding its product line to include different types of audio equipment such as headphones, speakers, and microphones. This will help attract a wider customer base and increase revenue.\n\n2. Enhance online presence: In today's digital age, having a strong online presence is crucial for any business. Soundwave Audio can invest in building a user-friendly website, creating social media accounts, and utilizing digital marketing strategies to reach a larger audience.\n\n3. Collabor"}


### Multiple Chains using Simple Sequential Chaining

In [12]:
from langchain.chains import SequentialChain

In [13]:
company_name_prompt = PromptTemplate.from_template("Suggest me a company name for making {product}")
company_name_chain = LLMChain(llm=client, prompt=company_name_prompt, output_key="company")

In [14]:
company_strategies_prompt = PromptTemplate.from_template("Suggest few strategies for the this company {company}")
company_strategies_chain = LLMChain(llm=client, prompt=company_strategies_prompt, output_key="strategies")

In [15]:
chain = SequentialChain(
    chains=[company_name_chain, company_strategies_chain],
    input_variables=["product"],
    output_variables=["company", "strategies"]
)

In [16]:
print(json.dumps(chain.invoke("orange juice"), indent=4))

{
    "product": "orange juice",
    "company": "\n\n1. Zestful Juicers\n2. Citrus Crush Co.\n3. Orange Oasis\n4. Sunkissed Sips\n5. Tangy Twist Co.\n6. Freshly Squeezed Co.\n7. Zesty Zing Co.\n8. Juice Me Up Co.\n9. Orange Harvest Co.\n10. The Juicy Orange Co.",
    "strategies": "\n\n1. Expand product line: Consider expanding the product line to include other types of juices such as apple, grapefruit, and pineapple. This will appeal to a wider range of customers and increase sales potential.\n\n2. Collaborate with local businesses: Partner with local cafes, restaurants, and grocery stores to sell your juices. This will increase brand awareness and reach a larger customer base.\n\n3. Create a loyalty program: Offer discounts or free products to customers who frequently purchase from your company. This will"
}
